# Load packages

In [2]:
# !pip install efficientnet
# !pip install pydot
# !pip install graphviz
# !pip install pydotplus
import tqdm
from tqdm import tqdm_notebook
import math
from PIL import Image
import pydot
import pandas as pd
import numpy as np
import os
import cv2
import tensorflow.keras.backend as K
import pydotplus
from pydotplus import graphviz
import pydot
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.utils.data_utils import Sequence
import efficientnet.tfkeras as efn
from keras.applications import VGG16

# BytesList = tf.train.BytesList
# FloatList = tf.train.FloatList
# Int64List = tf.train.Int64List
# Feature = tf.train.Feature
# Features = tf.train.Features
# Example = tf.train.Example

# to make this notebook's output stable across runs
# np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


print(f"Tensor flow version --> {tf.__version__}")
# from tensorflow.keras.utils import Sequence
print(f"Keras version --> {keras.__version__}")



OUTPUT_PATH = "/Users/paritoshgupta/Desktop/aamlp/aamlp/input/osic-pulmonary-fibrosis-progression/ads/"
OUTPUT_PATH_LUNG_MASK_IMAGES_1MM = "/Users/paritoshgupta/Desktop/aamlp/aamlp/input/osic-pulmonary-fibrosis-progression/ads/lung_mask_1mm/"


Tensor flow version --> 2.2.0
Keras version --> 2.3.0-tf


# UDF

In [3]:
# # C1, C2 = tf.constant(70, dtype='float32'), tf.constant(1000, dtype="float32")
# # def score(y_true, y_pred):
# #     tf.dtypes.cast(y_true, tf.float32)
# #     tf.dtypes.cast(y_pred, tf.float32)
# #     sigma = y_pred[:, 2] - y_pred[:, 0]
# #     fvc_pred = y_pred[:, 1]
    
# #     #sigma_clip = sigma + C1
# #     sigma_clip = tf.maximum(sigma, C1)
# #     delta = tf.abs(y_true[:, 0] - fvc_pred)
# #     delta = tf.minimum(delta, C2)
# #     sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32) )
# #     metric = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip* sq2)
# #     return keras.backend.mean(metric)

# # def qloss(y_true, y_pred):
# #     # Pinball loss for multiple quantiles
# #     qs = [0.2, 0.50, 0.8]
# #     q = tf.constant(np.array([qs]), dtype=tf.float32)
# #     e = y_true - y_pred
# #     v = tf.maximum(q*e, (q-1)*e)
# #     return keras.backend.mean(v)

# # def mloss(_lambda):
# #     def loss(y_true, y_pred):
# #         return _lambda * qloss(y_true, y_pred) + (1 - _lambda)*score(y_true, y_pred)
# #     return loss


# # def get_efficientnet(model, shape):
# #     models_dict = {
# #         'b0': efn.EfficientNetB0(input_shape=shape,weights=None,include_top=False),
# #         'b1': efn.EfficientNetB1(input_shape=shape,weights=None,include_top=False),
# #         'b2': efn.EfficientNetB2(input_shape=shape,weights=None,include_top=False),
# #         'b3': efn.EfficientNetB3(input_shape=shape,weights=None,include_top=False),
# #         'b4': efn.EfficientNetB4(input_shape=shape,weights=None,include_top=False),
# #         'b5': efn.EfficientNetB5(input_shape=shape,weights=None,include_top=False),
# #         'b6': efn.EfficientNetB6(input_shape=shape,weights=None,include_top=False),
# #         'b7': efn.EfficientNetB7(input_shape=shape,weights=None,include_top=False)
# #     }
# #     return models_dict[model]


# # # def build_model(input_shape_image, model_class = None):
# # #     image_inp = keras.layers.Input(shape=input_shape_image, name="input_image")
# # #     layer_model_head = get_efficientnet(model_class, input_shape_image)(image_inp)    
# # #     layer_model_head.trainable = False
# # #     layer_global_pooling = keras.layers.GlobalAveragePooling2D(name="pretrained_global_avg_pooling")(layer_model_head)
# # #     layer_global_pooling_out =keras.layers.Dense(320, activation="relu", name="global_pooling_out")(layer_global_pooling)
# # #     dropout_img_1 = keras.layers.Dropout(0.5, name="dropout_img_1")(layer_global_pooling_out)
# # # #     dense_layer_image_1 = keras.layers.Dense(64, activation="relu", name="global_pooling_out")(layer_global_pooling)
# # # #     dropout_img_2 = keras.layers.Dropout(0.3, name="dropout_img_2")(dense_layer_image_1)
# # #     out_image = keras.layers.Dense(16, activation ="relu", name ="output_class")(dropout_img_1)
    
# # #     tabular_inp= keras.layers.Input(shape=(8,))
# # #     concat = keras.layers.Concatenate(name="concat")([out_image, tabular_inp])
# # #     concat_out_1 = keras.layers.Dense(512, activation="relu", name="concat_out_1")(concat)
# # #     drop_out_img_tab_1 = keras.layers.Dropout(0.4, name="dropout_img_tab_1")(concat_out_1)
# # #     concat_out_2 = keras.layers.Dense(256, activation="relu", name="concat_out_2")(drop_out_img_tab_1)
# # #     drop_out_img_tab_2 = keras.layers.Dropout(0.3, name="dropout_img_tab_2")(concat_out_2)
# # #     concat_out_3 = keras.layers.Dense(100, activation="relu", name="concat_out_3")(drop_out_img_tab_2)
# # #     p1 = keras.layers.Dense(3, activation="linear", name="p1")(concat_out_3)
# # #     p2 = keras.layers.Dense(3, activation="relu", name="p2")(concat_out_3)
# # #     preds = keras.layers.Lambda(lambda x: x[0] + tf.cumsum(x[1], axis=1), 
# # #                      name="preds")([p1, p2])
    
# # #     model = keras.Model(inputs = [image_inp, tabular_inp], outputs = preds)
# # #     return model

# class DataGenerator(Sequence):
#     def __init__(self, dataset, batch_size=16,  shuffle=True):
#         'Initialization'
#         self.batch_size = batch_size
#         self.dataset = dataset
#         self.shuffle = shuffle
#         self.indexes = dataset.index
# #         self.on_epoch_end()
 
#     def __len__(self):
#         'Denotes the number of batches per epoch'
#         return math.ceil(len(self.dataset) / self.batch_size)
 
#     def __getitem__(self, index):
#         'Generate one batch of data'
#         # Generate indexes of the batch
#         idxs = [i for i in range(index*self.batch_size,(index+1)*self.batch_size)]
 
#         # Find list of IDs
#         ids_tmp = [self.indexes[k] for k in idxs]
        
#         # Generate data
#         # tabular
#         tab = self.dataset.loc[ids_tmp, ['Male', 'Female', 'Ex-smoker', 'Never smoked', 'Currently smokes', 'age', 'week', 'base_fvc']].to_numpy()#.reshape(-1)
        
#         # image
#         patient_ids = list(self.dataset.loc[ids_tmp, 'Patient'].to_numpy())
        
#         imgs_master = []
#         for pat_id in patient_ids:
#             images = os.listdir(os.path.join(OUTPUT_PATH_LUNG_MASK_IMAGES_1MM, pat_id))
#             sorted_images = sorted(images,key=lambda x: int(os.path.splitext(x)[0]))
#             slices = [Image.open(f"{OUTPUT_PATH_LUNG_MASK_IMAGES_1MM}/{pat_id}/{sorted_images[index]}").convert('L') for index, file in enumerate(sorted_images)]
#             tmp_array = np.mean([np.array(slices[i]) for i in range(int(len(slices)/4), int(len(slices)/2))], axis=0)
#             out = cv2.resize(tmp_array, dsize=(512, 512), interpolation=cv2.INTER_CUBIC)
#             imgs_master.append(out)
            
#         imgs_master = np.array(imgs_master)
#         imgs_master_out = np.expand_dims(imgs_master, axis=-1)
        
#         # target column
#         FVC = self.dataset.loc[ids_tmp, 'FVC'].to_numpy()
        
#     #print("u,i,r:", [User, Item],[y])
#         return [imgs_master_out, tab], FVC
 
#     def on_epoch_end(self):
#         'Updates indexes after each epoch'
#         self.indexes = np.arange(len(self.dataset))
#         if self.shuffle == True:
#             self.indexes = np.random.shuffle(self.indexes)

    
    
    

In [4]:
# model = build_model(input_shape_image = (512, 512, 1), model_class="b0")
# df_folds = pd.read_csv(f"{OUTPUT_PATH}/osic_folds.csv")

# def run(fold):
#     df_train = df_folds[df_folds.fold !=fold].reset_index(drop=True)
#     # get validation data
#     df_valid = df_folds[df_folds.fold==fold].reset_index(drop=True)
    
#     model.fit(DataGenerator(dataset=df_train), validation_data=DataGenerator(dataset=df_valid), verbose=1)
    
#     print("val", model.evaluate(DataGenerator(dataset=df_valid), verbose=1))

# Xception Model

In [5]:
ROT_ = 0
SHR_ = 2.0
HZOOM_ = 2
WZOOM_ = 4
HSHIFT_ = 2
WSHIFT_ = 4

## UDFs

In [39]:
# def build_model(input_shape_image):
    
#     input_image = keras.layers.Input(shape=input_shape_image, name="Image_Input")
    
#     base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False,
#                 input_tensor=keras.layers.Input(shape=input_shape_image))
#     base_model_output = base_model(input_image)
#     print(f"Number of layers in pre trained model --> {len(base_model.layers)}")
    
# #     base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)
# #     global average pooling 2d
#     layer_global_pooling = keras.layers.GlobalAveragePooling2D(name="pretrained_global_avg_pooling")(base_model_output)
    
# #     layer_global_pooling_out =keras.layers.Dense(32, activation="relu", name="global_pooling_out")(layer_global_pooling)
# #     dropout_img_1 = keras.layers.Dropout(0.3, name="dropout_img_1")(layer_next_flattenning)
# #     concat_out_1 = keras.layers.Dense(128, activation="relu", name="concat_out_1")(concat)
# #     drop_out_img_tab_1 = keras.layers.Dropout(0.3, name="dropout_img_tab_1")(concat_out_1)

#     # flatten, dropout, dense for image (Image Network)
# #     average_pooling_2d = keras.layers.GlobalAveragePooling2D(pool_size=(2,2), name="AvgPool2d")(base_model_output)
# #     layer_global_pooling = keras.layers.GlobalAveragePooling2D(name="pretrained_global_avg_pooling")(base_model_output)
# #     layer_flattening = keras.layers.Flatten(name="Flattening")(layer_global_pooling)
# #     layer_next_flattenning =keras.layers.Dense(128, activation="relu", name="Next_Flattening")(layer_global_pooling)
#     dropout_img_1 = keras.layers.Dropout(0.5, name="dropout_img_1")(layer_global_pooling)
# #     dropout_img_2 = keras.layers.Dropout(0.25, name="dropout_img_2")(layer_next_flattenning)
#     out_image = keras.layers.Dense(32, name ="output_class",activation="relu")(dropout_img_1)  
    
    
#     # Tabular Input
#     tabular_inp= keras.layers.Input(shape=(8,))
    
#     # Concat - Image and Tabular
#     concat = keras.layers.Concatenate(name="concat")([out_image, tabular_inp])
#     concat_out_1 = keras.layers.Dense(32, name="concat_out_2", activation="relu",
#                                      kernel_regularizer=keras.regularizers.l2(0.05))(concat)
#     drop_out_img_tab_1 = keras.layers.Dropout(0.2, name="dropout_img_tab_2")(concat_out_1)
#     concat_out_2 = keras.layers.Dense(16, name="concat_out_3", activation="relu")(drop_out_img_tab_1)
#     p1 = keras.layers.Dense(3, activation="linear", name="p1")(concat_out_2)
#     p2 = keras.layers.Dense(3, activation="relu", name="p2")(concat_out_2)
#     preds = keras.layers.Lambda(lambda x: x[0] + tf.cumsum(x[1], axis=1), 
#                      name="preds")([p1, p2])
    
#     model = keras.Model(inputs = [input_image, tabular_inp], outputs = preds)
#     for layer in base_model.layers:
#         layer.trainable = False
        
#     return base_model, model

def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))


def transform(image, image_size):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = image_size[0]
    XDIM = DIM%2 #fix for size 331
    
    rot = ROT_ * tf.random.normal([1], dtype='float32')
    shr = SHR_ * tf.random.normal([1], dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1], dtype='float32') / HZOOM_
    w_zoom = 1.0 + tf.random.normal([1], dtype='float32') / WZOOM_
    h_shift = HSHIFT_ * tf.random.normal([1], dtype='float32') 
    w_shift = WSHIFT_ * tf.random.normal([1], dtype='float32') 

    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    
    idx = tf.stack([x,y,z])    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)

    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3])


def score(y_true, y_pred):
    C1 = tf.constant(70, dtype='float32')
    C2 = tf.constant(1000, dtype="float32")
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    sigma = y_pred[:, 2] - y_pred[:, 0]
    fvc_pred = y_pred[:, 1]
    
    #sigma_clip = sigma + C1
    sigma_clip = tf.maximum(sigma, C1)
    delta = tf.abs(y_true[:, 0] - fvc_pred)
    delta = tf.minimum(delta, C2)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32) )
    metric = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip* sq2)
    return keras.backend.mean(metric)

def qloss(y_true, y_pred):
    # Pinball loss for multiple quantiles
    qs = [0.2, 0.50, 0.8]
    q = tf.constant(np.array([qs]), dtype=tf.float32)
    e = y_true - y_pred
    v = tf.maximum(q*e, (q-1)*e)
    return keras.backend.mean(v)

def mloss(_lambda):
    def loss(y_true, y_pred):
        return _lambda * qloss(y_true, y_pred) + (1 - _lambda)*score(y_true, y_pred)
    return loss

def convert_to_tensor(arg):
    out = tf.convert_to_tensor(arg, dtype=tf.float32)
    return out

def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")    

def plot_color_image(image):
    plt.imshow(image, interpolation="nearest")
    plt.axis("off")

def central_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]])
    top_crop = (shape[0] - min_dim) // 4
    bottom_crop = shape[0] - top_crop
    left_crop = (shape[1] - min_dim) // 4
    right_crop = shape[1] - left_crop
    return image[top_crop:bottom_crop, left_crop:right_crop]

def random_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]]) * 90 // 100
    return tf.image.random_crop(image, [min_dim, min_dim, 3])

def preprocess(image):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.vgg19.preprocess_input(resized_image*255)
    
    return final_image


class DataGenerator(Sequence):
    def __init__(self, dataset, batch_size, aug, shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.aug = aug
        self.dataset = dataset
        self.shuffle = shuffle
        self.indexes = np.arange(self.dataset.shape[0])
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.dataset.shape[0]/self.batch_size))
 
    def __getitem__(self, index):
        # Generate indexes of the batch
        idxs = [i for i in range(index*self.batch_size,(index+1)*self.batch_size)]
 
        # Find list of IDs
        ids_tmp = [self.indexes[k] for k in idxs]
        
        # Generate data
        # tabular
        tab = self.dataset.loc[ids_tmp, ['Male', 'Female', 'Ex-smoker', 'Never smoked', 
                                         'Currently smokes', 'age', 'week', 'base_fvc']].to_numpy(dtype=np.float32)
        
        # image
        patient_ids = list(self.dataset.loc[ids_tmp, 'Patient'])
        imgs_master = []
        
        for pat_id in patient_ids:    
            images = os.listdir(os.path.join(OUTPUT_PATH_LUNG_MASK_IMAGES_1MM, pat_id))
            sorted_images = sorted(images,key=lambda x: int(os.path.splitext(x)[0]))
            slices = [Image.open(f"{OUTPUT_PATH_LUNG_MASK_IMAGES_1MM}/{pat_id}/{sorted_images[index]}").convert('L') for index, file in enumerate(sorted_images)]    
            
            
            ######## OLD ########
#             img1 = np.mean([np.array(slices[i]) for i in range(int(0.15 * len(slices)), int(0.4 * len(slices)))], axis=0)/255
#             img1 = tf.convert_to_tensor(img1.reshape((*img1.shape, 1)))
#             img2 = np.mean([np.array(slices[i]) for i in range(int(0.4 * len(slices)/2), int(0.65 * len(slices)))], axis=0)/255
#             img2 = tf.convert_to_tensor(img2.reshape((*img2.shape, 1)))
#             img3 = np.mean([np.array(slices[i]) for i in range(int(0.65 * len(slices)), int(0.9 * len(slices)))], axis=0)/255
#             img3 = tf.convert_to_tensor(img3.reshape((*img3.shape, 1)))

#             randint = np.random.randint(0,3)
#             if randint == 0:
#                 img_to_resize = img1
#             elif randint == 1:
#                 img_to_resize = img2
#             else:
#                 img_to_resize = img3

            ######## NEW Augementation (Chris Deotte NB) ########
            index = int(len(slices)/3.33)
            img1 = np.array(slices[index])/255
            img_to_resize = tf.convert_to_tensor(img1.reshape((*img1.shape, 1)))
            
            images = np.array([img_to_resize]*3)
            images_reduced_mean = tf.reduce_mean(images, axis=3)
            image_pre_proc = tf.stack([images_reduced_mean[0], images_reduced_mean[1], images_reduced_mean[2]], axis=-1)

            # With Augmentation (Training time)
            if self.aug:
                trans_image = transform(image_pre_proc, image_size = image_pre_proc.shape)
                resized_image = preprocess(trans_image)[:, :, 0]
            else:
                resized_image = preprocess(image_pre_proc)[:, :, 0]
                
            out_img = tf.stack([resized_image, resized_image, resized_image], axis=-1)
            imgs_master.append(out_img)
                        
        imgs_master_out = tf.stack(imgs_master)

        # target column
        FVC = self.dataset.loc[ids_tmp, 'FVC'].to_numpy(dtype=np.float32)

        # to tensor convert
        FVC = convert_to_tensor(tf.constant(FVC))
        tab = convert_to_tensor(tf.constant(tab))
        
        return [imgs_master_out, tab], FVC
 
    def on_epoch_end(self):
        self.indexes = np.arange(self.dataset.shape[0])
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
#         if self.shuffle:
#             np.random.shuffle(self.indexes)


##  Model

In [40]:
# resnet_model = keras.applications.xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
#     base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)
#     global average pooling 2d
# layer_global_pooling = keras.layers.GlobalAveragePooling2D(name="pretrained_global_avg_pooling")(base_model_output)
#     layer_global_pooling_out =keras.layers.Dense(32, activation="relu", name="global_pooling_out")(layer_global_pooling)
#     dropout_img_1 = keras.layers.Dropout(0.3, name="dropout_img_1")(layer_next_flattenning)
#     concat_out_1 = keras.layers.Dense(128, activation="relu", name="concat_out_1")(concat)
#     drop_out_img_tab_1 = keras.layers.Dropout(0.3, name="dropout_img_tab_1")(concat_out_1)

# flatten, dropout, dense for image (Image Network)
#     average_pooling_2d = keras.layers.GlobalAveragePooling2D(pool_size=(2,2), name="AvgPool2d")(base_model_output)
#     layer_global_pooling = keras.layers.GlobalAveragePooling2D(name="pretrained_global_avg_pooling")(base_model_output)
#     layer_flattening = keras.layers.Flatten(name="Flattening")(layer_global_pooling)
#     layer_next_flattenning =keras.layers.Dense(128, activation="relu", name="Next_Flattening")(layer_global_pooling)
# dropout_img_1 = keras.layers.Dropout(0.5, name="dropout_img_1")(layer_global_pooling)


### Summary

In [47]:
input_shape_image=(224,224,3)
input_image = keras.layers.Input(shape=input_shape_image, name="Image_Input")

########## Im,age Input ##########
# base_model = keras.applications.VGG19(weights="imagenet", include_top=False,
#             input_tensor=keras.layers.Input(shape=input_shape_image))
base_model = keras.applications.xception.Xception(weights='imagenet', include_top=False, input_shape=input_shape_image)
base_model.trainable=False
base_model_output = base_model(input_image, training=False)
layer_global_pooling = keras.layers.GlobalAveragePooling2D(name="pretrained_global_avg_pooling")(base_model_output)
out_dense_1 = keras.layers.Dense(64, name ="output_dense_img_1",activation="relu")(layer_global_pooling)  
bn_out_1 = keras.layers.BatchNormalization()(out_dense_1)
dropout_img_1 = keras.layers.Dropout(0.5, name="dropout_img_1")(bn_out_1)
out_dense_2 = keras.layers.Dense(16, name ="output_dense_img_2",activation="relu")(dropout_img_1)  
# out_image = keras.layers.Dense(16, name ="output_image",activation="relu")(dropout_img_2)  

########## Tabular Input ##########
tabular_inp= keras.layers.Input(shape=(8,))
# Concat - Image and Tabular
concat = keras.layers.Concatenate(name="concat")([out_dense_2, tabular_inp])
concat_out_1 = keras.layers.Dense(64, name="concat_out_2", activation="relu",
                                 kernel_regularizer=keras.regularizers.l2(0.1)
                                 )(concat)
bn_out_2 = keras.layers.BatchNormalization()(concat_out_1)
drop_out_img_tab_1 = keras.layers.Dropout(0.25, name="dropout_img_tab_2")(bn_out_2)
concat_out_2 = keras.layers.Dense(32, name="concat_out_3", activation="relu")(drop_out_img_tab_1)
p1 = keras.layers.Dense(3, activation="linear", name="p1")(concat_out_2)
p2 = keras.layers.Dense(3, activation="relu", name="p2")(concat_out_2)
preds = keras.layers.Lambda(lambda x: x[0] + tf.cumsum(x[1], axis=1), 
                 name="preds")([p1, p2])

model = keras.Model(inputs = [input_image, tabular_inp], outputs = preds)
for layer in base_model.layers:
    layer.trainable = False

print(model.summary())
print(f" N Layers of BASE model --> {len(base_model.layers)}")
print(f" N Layers of our model --> {len(model.layers)}")
for index, layer in enumerate(model.layers):
    print(index, layer.name)

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Image_Input (InputLayer)        [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
xception (Model)                (None, 7, 7, 2048)   20861480    Image_Input[0][0]                
__________________________________________________________________________________________________
pretrained_global_avg_pooling ( (None, 2048)         0           xception[1][0]                   
__________________________________________________________________________________________________
output_dense_img_1 (Dense)      (None, 64)           131136      pretrained_global_avg_pooling[0][
____________________________________________________________________________________________

In [51]:
base_model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_19[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [49]:
trainable = 4
base_model.trainable=True
print(f"N layers in base model --> {len(base_model.layers)}")
print(f"Freezing Top {trainable} layers")
for layer in base_model.layers[:(len(base_model.layers) - trainable)]:
        layer.trainable = False
for layer in base_model.layers[(len(base_model.layers) - trainable):]:
    layer.trainable = True


N layers in base model --> 132
Freezing Top 4 layers


In [50]:
optimizer = keras.optimizers.SGD(lr=1e-5, momentum=0.9)
model.compile(loss=mloss(0.8), optimizer=optimizer, metrics=[score])
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Image_Input (InputLayer)        [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
xception (Model)                (None, 7, 7, 2048)   20861480    Image_Input[0][0]                
__________________________________________________________________________________________________
pretrained_global_avg_pooling ( (None, 2048)         0           xception[1][0]                   
__________________________________________________________________________________________________
output_dense_img_1 (Dense)      (None, 64)           131136      pretrained_global_avg_pooling[0][
____________________________________________________________________________________________

In [42]:
def get_lr_callback():
    lr_start   = 0.1
    lr_max     = 0.1
    lr_min     = 0.00001
    lr_ramp_ep = 10
    lr_sus_ep  = 0
    lr_decay   = 0.8
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
            
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max
            
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
            
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=False)
    return lr_callback

### Compile for fold = 0 

In [43]:
# def run(model, fold):
fold = 0
    # read folds data and get train & validation data 
df_folds = pd.read_csv(f"{OUTPUT_PATH}/osic_10_folds.csv")
df_train = df_folds[df_folds.fold !=fold].reset_index(drop=True)
df_train = df_train.sample(frac=1).reset_index(drop=True)
print(f"Train Shape --> {df_train.shape}")
print(f"Number of unique patients in Train --> {df_train.Patient.nunique()}")

# get validation data
df_valid = df_folds[df_folds.fold==fold].reset_index(drop=True)  
print(f"Validation Shape --> {df_valid.shape}")
print(f"Number of unique patients in Validation --> {df_valid.Patient.nunique()}")

# compile model
# lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=1)
optimizer = keras.optimizers.Adam(lr=0.05)
model.compile(loss=mloss(0.8), 
          optimizer=optimizer, metrics=[score])

# check point and early stopping
checkpoint_cb = keras.callbacks.ModelCheckpoint(f"/Users/paritoshgupta/Desktop/osic_models/osic_fold_{fold}.h5",
                                                save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

# fit model 

history = model.fit(DataGenerator(dataset=df_train, batch_size=32, aug=True, shuffle=False),
                    validation_data=DataGenerator(dataset=df_valid, batch_size=32, aug=False, shuffle=False), 
                    epochs=5,
                    callbacks=[checkpoint_cb, early_stopping_cb, get_lr_callback()])
# save model 
model.save(f"/Users/paritoshgupta/Desktop/osic_models/test_model.h5")

# print validation data score
print(f"Validation Data Score --> {model.evaluate(DataGenerator(dataset=df_valid, batch_size=16, aug=False, shuffle=False))}")

# return model via UDF
# return model

Train Shape --> (1339, 11)
Number of unique patients in Train --> 153
Validation Shape --> (143, 11)
Number of unique patients in Validation --> 16
Epoch 1/5
41/41 [==============================] - 1746s 43s/step - loss: 395.8427 - score: 9.3513 - val_loss: 205.9217 - val_score: 8.3790 - lr: 0.1000
Epoch 2/5
41/41 [==============================] - 1782s 43s/step - loss: 271.2944 - score: 8.3280 - val_loss: 220.1334 - val_score: 8.0426 - lr: 0.1000
Epoch 3/5
8/8 [==============================] - 94s 12s/step - loss: 205.9217 - score: 8.3790
Validation Data Score --> [205.92173767089844, 8.37901782989502]


In [44]:
df_test = pd.read_csv(f"{OUTPUT_PATH}/osic_test.csv")
print(df_test.shape)
# df_test
print(f"Test Data Score --> {model.evaluate(DataGenerator(dataset=df_test, batch_size=4, aug=False, shuffle=False))}")


(44, 10)
11/11 [==============================] - 57s 5s/step - loss: 180.4018 - score: 8.3044
Test Data Score --> [180.40184020996094, 8.304366111755371]


In [42]:
y_output_single = y_output.mean(axis=0)
y_output_single

array([[1120.6364, 1379.5336, 2179.7068],
       [1448.4658, 1782.8522, 2817.1238],
       [1407.4906, 1732.5883, 2737.5574],
       [1386.048 , 1706.1971, 2695.9404],
       [1540.7133, 1896.422 , 2996.5154],
       [1672.2847, 2058.2146, 3251.9846],
       [1456.744 , 1793.103 , 2833.614 ],
       [1310.0302, 1612.6813, 2548.713 ],
       [1684.0947, 2072.8123, 3275.2063],
       [1651.7823, 2032.9554, 3212.5076],
       [1816.819 , 2236.1318, 3533.666 ],
       [1429.441 , 1759.4077, 2780.2961]], dtype=float32)

In [52]:
# y_output_single.shape
np.array(df_test.FVC)

array([2315., 2214., 2061., ..., 3364., 3240., 3303.])

In [45]:
score([df_test.FVC], [y_output_single])

TypeError: list indices must be integers or slices, not tuple

### Run Model 

In [ ]:
# model = run(model, 0)


In [ ]:
# Fine-tune from this layer onwards
fine_tune_at = 153

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[fine_tune_at:]:
    layer.trainable = True
    
optimizer = keras.optimizers.Adam(lr=0.0025, beta_1=0.9, beta_2=0.999)

model.compile(loss=mloss(0.8), 
          optimizer=optimizer, metrics=[score])
model.summary()



In [ ]:
history_fine = model.fit(DataGenerator(dataset=df_train),
                    validation_data=DataGenerator(dataset=df_valid), epochs=20,
                    callbacks=[checkpoint_cb, early_stopping_cb],
                   initial_epoch=history.epoch[-1])
print(f" Validation Data Score --> {model.evaluate(DataGenerator(dataset=df_valid))}")


## Plot History

In [ ]:
import pandas as pd

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
save_fig("keras_learning_curves_plot")
plt.show()

###  Load model 

In [ ]:
os.listdir("/Users/paritoshgupta/Desktop/osic_models/")

# model = keras.models.load_model("my_keras_model.h5")


In [ ]:
run(0)

# Playground

In [ ]:
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
Image_Input (InputLayer)        [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
xception (Model)                (None, 7, 7, 2048)   20861480    Image_Input[0][0]                
__________________________________________________________________________________________________
pretrained_global_avg_pooling ( (None, 2048)         0           xception[1][0]                   
__________________________________________________________________________________________________
dropout_img_1 (Dropout)         (None, 2048)         0           pretrained_global_avg_pooling[0][
__________________________________________________________________________________________________
output_class (Dense)            (None, 32)           65568       dropout_img_1[0][0]              
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
concat (Concatenate)            (None, 40)           0           output_class[0][0]               
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
concat_out_2 (Dense)            (None, 64)           2624        concat[0][0]                     
__________________________________________________________________________________________________
dropout_img_tab_2 (Dropout)     (None, 64)           0           concat_out_2[0][0]               
__________________________________________________________________________________________________
concat_out_3 (Dense)            (None, 32)           2080        dropout_img_tab_2[0][0]          
__________________________________________________________________________________________________
p1 (Dense)                      (None, 3)            99          concat_out_3[0][0]               
__________________________________________________________________________________________________
p2 (Dense)                      (None, 3)            99          concat_out_3[0][0]               
__________________________________________________________________________________________________
preds (Lambda)                  (None, 3)            0           p1[0][0]                         
                                                                 p2[0][0]                         
============================================

In [46]:
# len(base_model.layers)
# base_model.summary()
# keras.optimizers.RMSprop()

0.001/1e-4

# 1e-4



10.0

In [60]:
# len(base_model.layers)
model.compile

155

In [70]:
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_39 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_39[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [71]:
resnet_model.layers

In [30]:
a = [1,2,43,5]
np.random.shuffle(a)

In [31]:
a

[2, 1, 5, 43]

In [ ]:
keras.applications.xception.Xception(weights="imagenet", include_top=False,
                input_tensor=keras.layers.Input(shape=input_shape_image))